In [1]:
# main_script.py

# --- 1. Import necessary libraries ---
import torch
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import os

# --- 2. Data Generation and Loading ---
# This section generates a synthetic dataset for binary classification
# and saves it to a CSV file if it doesn't already exist.

def generate_or_load_data(filename='binary_data.csv'):
    """
    Generates a synthetic binary classification dataset if the file doesn't exist.
    Loads the data from the CSV file into a pandas DataFrame.
    """
    if not os.path.exists(filename):
        print(f"'{filename}' not found. Generating a new dataset.")
        # Generate a dataset with 100 samples, 2 input features, and 2 classes.
        X, y = make_classification(
            n_samples=100,
            n_features=2,
            n_informative=2,
            n_redundant=0,
            n_classes=2,
            random_state=1
        )
        # Create a DataFrame and save it to CSV.
        df = pd.DataFrame(X, columns=['feature_1', 'feature_2'])
        df['label'] = y
        df.to_csv(filename, index=False)
        print(f"Dataset saved to '{filename}'.")
    else:
        print(f"Loading existing dataset from '{filename}'.")

    # Load the data from the CSV file.
    return pd.read_csv(filename)

# --- 3. Model, Loss, and Training Functions (Manual Implementation) ---
# We define our functions from scratch to avoid using torch.nn.

def sigmoid(z):
    """Sigmoid activation function."""
    return 1 / (1 + torch.exp(-z))

def binary_cross_entropy_loss(y_true, y_pred):
    """
    Binary Cross-Entropy loss function.
    We add a small epsilon value to prevent log(0) which results in NaN.
    """
    epsilon = 1e-7
    # Clamp predictions to avoid log(0) or log(1) issues
    y_pred = torch.clamp(y_pred, epsilon, 1 - epsilon)
    # BCE formula
    loss = -torch.mean(y_true * torch.log(y_pred) + (1 - y_true) * torch.log(1 - y_pred))
    return loss

# --- 4. Main Execution Block ---
if __name__ == "__main__":
    # --- Data Preparation ---
    # Load data and get features (X) and labels (y)
    data_df = generate_or_load_data()
    X = data_df[['feature_1', 'feature_2']].values
    y = data_df['label'].values

    # Split data into training (80%) and testing (20%) sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # --- Convert to PyTorch Tensors and Move to Device ---
    # Set device to GPU if available, otherwise CPU
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"\nUsing device: '{device}'")

    # Convert numpy arrays to PyTorch tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1).to(device)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1).to(device)

    # --- Hyperparameters ---
    n_features = X_train_tensor.shape[1]
    learning_rate = 0.1
    epochs = 100

    # --- Model Initialization (Manual) ---
    # Initialize weights and bias.
    # We set requires_grad=True to enable automatic gradient computation.
    weights = torch.randn(n_features, 1, device=device, requires_grad=True, dtype=torch.float32)
    bias = torch.zeros(1, device=device, requires_grad=True, dtype=torch.float32)

    print("\n--- Starting Training ---")
    # --- Training Loop ---
    for epoch in range(epochs):
        # --- Forward Pass ---
        # 1. Calculate the linear combination (Y = w^T * X + b)
        linear_output = X_train_tensor @ weights + bias
        # 2. Apply the sigmoid activation function
        y_pred = sigmoid(linear_output)

        # --- Calculate Loss ---
        loss = binary_cross_entropy_loss(y_train_tensor, y_pred)

        # --- Backward Pass ---
        # PyTorch automatically calculates the gradients of the loss
        # with respect to the tensors that have requires_grad=True (weights and bias).
        loss.backward()

        # --- Manual Weight Update (Gradient Descent) ---
        # We wrap this in torch.no_grad() because we don't want to track
        # this operation in the computation graph.
        with torch.no_grad():
            weights -= learning_rate * weights.grad
            bias -= learning_rate * bias.grad

            # --- Zero the Gradients ---
            # It's crucial to zero the gradients after each update,
            # otherwise they will accumulate on subsequent backward passes.
            weights.grad.zero_()
            bias.grad.zero_()

        # Print loss every 10 epochs
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item():.4f}")

    print("--- Training Finished ---\n")

    # --- Evaluation on Test Set ---
    with torch.no_grad():
        # Make predictions on the test data
        test_linear_output = X_test_tensor @ weights + bias
        test_pred_probs = sigmoid(test_linear_output)

        # Convert probabilities to binary predictions (0 or 1) by thresholding at 0.5
        test_pred_labels = (test_pred_probs >= 0.5).float()

        # Calculate accuracy
        correct_predictions = (test_pred_labels == y_test_tensor).sum().item()
        total_samples = len(y_test_tensor)
        accuracy = (correct_predictions / total_samples) * 100
        print(f"Accuracy on test set: {accuracy:.2f}%")

'binary_data.csv' not found. Generating a new dataset.
Dataset saved to 'binary_data.csv'.

Using device: 'cpu'

--- Starting Training ---
Epoch 10/100, Loss: 0.8364
Epoch 20/100, Loss: 0.5942
Epoch 30/100, Loss: 0.4539
Epoch 40/100, Loss: 0.3695
Epoch 50/100, Loss: 0.3151
Epoch 60/100, Loss: 0.2775
Epoch 70/100, Loss: 0.2501
Epoch 80/100, Loss: 0.2291
Epoch 90/100, Loss: 0.2126
Epoch 100/100, Loss: 0.1992
--- Training Finished ---

Accuracy on test set: 100.00%
